# DU IA et Santé - Atelier NLP

Dans cet atelier, nous allons voir plusieurs techniques de NLP "moderne", principalement basées sur les transformers et la librairie [huggingface](https://huggingface.co/).

Cet atelier est diviser en trois partie.

Dans la partie 1, nous allons nous intéresser à la partie encodage des transformers (BERT et cie.) afin de mieux comprendre les représentations internes de ceux-ci.

Dans la partie 2, nous interesserons à la partie decodage des transformers (GPT et cie.) et aux méthodes de *prompt engineering* permettant d’améliorer leurs résultats.

Enfin, dans la partie 3, nous allons voir comment méler les deux approches pour construire des assistants personnels.

Mais, avant toute choses, il nous faut installer quelques librairies.

Pour cela, executez la cellule suivante:

In [ ]:
!pip install accelerate jupyter-scatter

Puis, nous allons importer quelques librairies et vérifier que nous accédons bien au GPU (si vous en avez un).

Pour cela exécutez la cellule ci-dessous.

In [ ]:
# Librairies utilitaires
from tqdm.auto import tqdm
import ipywidgets
import gc

# Librairies mathématiques
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline
import jscatter

# Librairies ML
import torch
print("cuda available?", str(torch.cuda.is_available()))
device = torch.device("cpu")
if torch.cuda.is_available():
  device = torch.device("cuda:0")
  torch.cuda.set_device(device)
  print("cuda version:", torch.version.cuda)
  print("cuDNN enabled?", torch.backends.cudnn.enabled)
  print("cuDNN version:", torch.backends.cudnn.version())
  print("Device name? ", torch.cuda.get_device_name(torch.cuda.current_device()))

# Initialisation générateurs de nombres aléatoires
SEED = 42
np.random.seed(SEED)
torch.random.manual_seed(SEED)

Si le résultat est : `cuda available? False`, cela signifie que la librairie *torch* n’a pas pu accéder à votre GPU.

Dans le cas contraire (ou si vous n’avez pas de GPU à disposition), nous pouvons passer à la suite.

remarque: si vous utiliser ce GoogleCollab, cliquez sur Exécution > Modifier le type d’exécution et séléctionnnez l’option T4 GPU

## Partie 1 - Encodage et représentations internes

Dans cette partie, nous allons voir comment les transformers, plus spécifiquement les modèles de type BERT, encodent et transforment les textes fournis pour les donner en entrée des modèles. Nous allons aussi étudier les représentations internes de ce modèles, leurs espaces latents, et comment manipuler ces réprésentations.

Pour cela, nous allons nous baser sur le modèle [CamemBERT](https://camembert-model.fr/), entrainé sur des textes en français.

Pour essayer un autre modèle, voir: https://huggingface.co/models

In [ ]:
from transformers import AutoModel, AutoTokenizer

model_name = "Lajavaness/sentence-camembert-base"

model = AutoModel.from_pretrained(model_name, device_map=device, torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)

### 1.1 Tokenization et encodage

La première étape pour permettre à un modèle de language de traiter du texte est la tokenisation.

Cette étape permet de découper une phrase ou un mot en un ou plusieurs *token* connus du modèle (l’ensemble des tokens connus par un modèle est appelé son "vocabulaire").

Par exemple:

In [ ]:
sentence = "fromage"
tokenized_sentence = tokenizer.tokenize(sentence)
tokenized_sentence

Cependant, un modèle attend en entrée non pas une liste de tokens, mais la liste des id correspondant à ces tokens dans son vocabulaire.

Ainsi, il nous faut encoder les tokens obtenus précédemment en une liste d’entier utilisable par le modèle.

In [ ]:
encoded_sentence = tokenizer.encode(tokenized_sentence, is_split_into_words=True, return_tensors="pt")
encoded_sentence

Le résultat ci-dessus présente la liste des ids correspondant aux tokens de notre phrase.

Nous pouvons cependant noter la précense de deux ids supplémentaires (5 et 6), ceux-ci correspondants aux tokens de début et fin de phrase.

Sentez-vous libre de modifier la phrase d’exemple avant de passer à la suite.

### 1.2 Word Embeddings et espaces latents

Maintenant que nous avons comment tokeniser et encoder un texte pour permettre à un modèle de manipuler ce texte, nous allons voir les réprentations internes utilisé par ces modèles pour manipuler ces textes.

Pour cela nous allons utiliser la classe *pipeline* de la librairie *transformers*.

In [7]:
from transformers import pipeline

Cette classe permet d’utiliser nos modèles sur divers problèmes.

Ici, nous allons l’utiliser pour effectuer de la *feature-extraction* et ainsi obtenir les représentations interne des phrases fournies à notre modèle.

In [8]:
pipeline = pipeline('feature-extraction', model=model, tokenizer=tokenizer)

Par exemple:

In [ ]:
data = pipeline(sentence)
pd.DataFrame(data[0])

In [ ]:
pd.DataFrame([np.array(data[0]).mean(axis=0)])

Nous voyons ici, que notre modèle transforme chaque token de notre phrase en vecteurs de taille:

In [ ]:
len(data[0][1])

La taille de ces vecteurs, générallement appellés *embeddings*, determine le nombre de dimensions de l’espace latent de notre modèle.

Chaque vecteur correspondant alors à un point dans cet espace.

Il est alors possible d’avoir un aperçu du vocabulaire d’un modèle dans son espace latent.

Pour cela, il nous faut dans un premier temps recupérer les embeddings de chaque token présent dans le vocabulaire d’un modèle.

In [ ]:
# On récupère le vocabulaire du modèle
vocab = [token.replace("▁", "") for token in list(tokenizer.get_vocab().keys())]

# On encodage le vocabulaire
embedded_vocab = pipeline(vocab, batch_size=32)

# On met le résultat sous un format plus facile à utiliser
df_embedded_vocab = pd.DataFrame([np.array(ev[0]).mean(axis=0) for ev in embedded_vocab], index=vocab)

# On affiche le résultat
df_embedded_vocab

On peut alors récupérer l’embedding d’un token specifique, par exemple:

In [ ]:
df_embedded_vocab.loc[[sentence]]

In [16]:
from sklearn.decomposition import PCA

In [17]:
pca = PCA(n_components=2)
pca_embedded_vocab = pca.fit_transform(df_embedded_vocab)

In [18]:
df_pca = pd.DataFrame(data = pca_embedded_vocab, columns = ['PC1', 'PC2'], index=vocab)

In [ ]:
scatter = jscatter.Scatter(data=df_pca, x="PC1", y="PC2")

output = ipywidgets.Output()

@output.capture(clear_output=True)
def selection_change_handler(change):
  display(df_pca.iloc[change.new])

scatter.widget.observe(selection_change_handler, names=["selection"])

ipywidgets.HBox([scatter.show(), output])

Example de visualisation:

https://projector.tensorflow.org/

https://helboukkouri.github.io/embedding-visualization/

Chaque dimension correspond à un "concept" utilisé par le modèle pour différencier les *tokens* entre eux.

Chaque cellule d’un *embedding* correspond alors à la position, allant de -1 à 1, du token associé sur une des dimensions de l’espace latent du modèle.

Nous pouvons, par exemple, visualiser la position d’un token sur chaque dimension à l’aide d’une *heatmap*.

In [ ]:
# On commence par convertir notre embedding dans un format plus adapté
df = df_embedded_vocab.iloc[[5271]]

# Et on lance la représentation (des 10 premières valeurs pour plus de lisibilité)
# sous forme de heatmap
sb.heatmap(df.iloc[:, : 10], vmin=-1.0, vmax=1.0, annot=True, fmt=".2f")

In [ ]:
### Exemples token proches
from sklearn.metrics.pairwise import cosine_similarity

tst = df_embedded_vocab.loc[[sentence]]

df_tst = pd.DataFrame(cosine_similarity(tst, df_embedded_vocab)[0], columns=["similarity"], index=vocab)
df_tst = df_tst.sort_values(by=['similarity'], ascending=False)

df_tst

Voir:

https://degaucheoudedroite.delemazure.fr/

In [ ]:
### TODO Exemples math sur vecteur (ROI - HOMME + FEMME)
mot1 = pipeline("France")
mot1 = pd.DataFrame([np.array(mot1[0]).mean(axis=0)])
mot1

In [ ]:
mot2 = pipeline("Camembert")
mot2 = pd.DataFrame([np.array(mot2[0]).mean(axis=0)])
mot2

In [ ]:
mot3 = pipeline("Mozzarella")
mot3 = pd.DataFrame([np.array(mot3[0]).mean(axis=0)])
mot3

In [ ]:
tst = mot1 - mot2 + mot3
tst

In [ ]:
df_tst = pd.DataFrame(cosine_similarity(tst, df_embedded_vocab)[0], columns=["similarity"], index=vocab)
df_tst = df_tst.sort_values(by=['similarity'], ascending=False)

df_tst.iloc[:10]

Voir: http://nlp.polytechnique.fr/word2vec

https://neal.fun/infinite-craft/

In [ ]:
del embedded_vocab
del df_embedded_vocab
del vocab
gc.collect()

Pour en savoir plus sur le fonctionnement des embeddings, voir: https://jalammar.github.io/illustrated-word2vec/

### 1.3 Sauvegarde et recherche sémantique de documents

In [ ]:
data = pipeline("J'aime le fromage")
pd.DataFrame(data[0])

In [ ]:
data1 = pipeline("J'aime le fromage",return_tensors = "pt")[0].numpy().mean(axis=0)
data1 = pd.DataFrame([data1])
data1

In [ ]:
data2 = pipeline("Je déteste le fromage",return_tensors = "pt")[0].numpy().mean(axis=0)
data2 = pd.DataFrame([data2])
data2

In [ ]:
data3 = pipeline("Le fromage, c'est vraiment bon", return_tensors = "pt")[0].numpy().mean(axis=0)
data3 = pd.DataFrame([data3])
data3

In [ ]:
data4 = pipeline("Le soleil brille sur la ville de Paris aujourd'hui", return_tensors = "pt")[0].numpy().mean(axis=0)
data4 = pd.DataFrame([data4])
data4

In [ ]:
cosine_similarity(data1, data2)

In [ ]:
cosine_similarity(data1, data3)

In [ ]:
cosine_similarity(data1, data4)

In [61]:
bdd_embeddings = pd.DataFrame(columns=data1.columns)
bdd_texts = []

def store_text_embeddings(text):
  if not text in bdd_texts:
    embeddings = pipeline(text, return_tensors = "pt")[0].numpy().mean(axis=0)
    bdd_embeddings.loc[len(bdd_embeddings.index)] = embeddings
    bdd_texts.append(text)

In [62]:
texts = [
"Camembert est une appellation générique qui désigne généralement un fromage à pâte molle et à croûte fleurie. Commercialement, cette appellation d'origine normande ne fait l'objet d'aucune protection et se voit utilisée pour des fromages n'ayant parfois que peu de rapport avec le camembert originel. Dans certaines régions de France, le camembert est appelé « claquos », « clacos », « calendos ».",
"Le bleu d’Auvergne est un fromage à pâte persillée fabriqué en France dans le Massif central à partir de lait de vache. Son persillage allant du bleu au bleu noir. Son appellation d'origine bénéficie de protections depuis 1975.",
"La tomme de Savoie est un fromage produit en France dans la région alpine de Savoie, regroupant les départements de la Savoie et de la Haute-Savoie. Son appellation est protégée par une indication géographique protégée.",
"Le munster ou munster-géromé (ou encore Minschterkäs en francique lorrain ou Minschterkaas en alsacien) est un fromage à pâte molle fabriqué à partir de lait de vache dans l'Est de la France. Son appellation est protégée nationalement depuis 1969 par une appellation d'origine contrôlée (AOC) et dans l'ensemble des pays de l'Union européenne depuis 1996 par une appellation d'origine protégée (AOP).",
"Les bries sont une famille de fromages à pâte molle à croûte fleurie, originaire de la région française de Brie.",
"L’emmentaler ou emmental est un fromage d'origine suisse à pâte dure dont le nom provient de la vallée de l'Emme (en allemand, Emmental), une région à l'est du canton de Berne.",
"Le gorgonzola est l'appellation d'origine d'un fromage traditionnel à base de lait de vache, à pâte persillée, fabriqué dans les régions de Lombardie et du Piémont.",
"Le Valençay est une appellation d'origine désignant un fromage de chèvre au lait cru du Berry, et de la région Centre-Val de Loire, en France. Elle reprend le nom de la commune homonyme.",
"Le saint-marcellin IGP est un fromage français du Dauphiné. Son Indication géographique protégée (IGP) date de la fin 2013, elle s'étend sur 274 communes en Isère, dans la Drôme et en Savoie.",
"Reblochon ou reblochon de Savoie est une appellation d'origine désignant un fromage français produit principalement en Haute-Savoie et dans quelques communes de Savoie. Cette appellation est originaire du massif des Bornes et des Aravis, principalement la vallée de Thônes, et s'est étendue au val d'Arly et au massif des Bauges.",
"Comté est l'appellation d'origine d'un fromage français transformé principalement en Franche-Comté et bénéficiant d'une AOC depuis 1958 et d'une AOP depuis 1996. Son aire de production s'étend dans les départements du Jura, du Doubs, et de l'est de l'Ain. Elle englobe également une commune de Haute-Savoie et quelques-unes de Saône-et-Loire."
]

for text in texts:
  store_text_embeddings(text)

In [ ]:
bdd_embeddings

In [ ]:
bdd_texts

In [82]:
def find_documents(query, nresults=5):
  query_embeddings = pd.DataFrame([pipeline(query, return_tensors = "pt")[0].numpy().mean(axis=0)])
  df = pd.DataFrame(cosine_similarity(query_embeddings, bdd_embeddings)[0], columns=["similarity"])
  df = df.sort_values(by=['similarity'], ascending=False)

  indexes = df.iloc[:nresults].index

  results = []
  for i in indexes:
    results.append({"doc": bdd_texts[i], "score": df.loc[i]["similarity"]})

  return results

In [ ]:
query = """
Fromage de Savoie
"""

find_documents(query)

Voir: https://www.trychroma.com/

## Partie 2 - Large modèles de langage et techniques de génération de texte

In [85]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [ ]:
model_name = "microsoft/Phi-3-mini-128k-instruct"

model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-mini-128k-instruct", device_map=device, torch_dtype="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")


### 2.1 Complétion de texte

TODO

In [87]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

In [141]:
generation_args = {
    "max_new_tokens": 1,
    "return_full_text": False,
    "do_sample": False
}

In [144]:
prompt = "Le fromage c'est vraiment bon, surtout le"

In [ ]:
output = pipe(prompt, **generation_args)
print(output[0]['generated_text'])

In [147]:
gereration_args["num_return_sequences"] = 3

In [ ]:
output = pipe(prompt, **generation_args)
print(output)

In [152]:
generation_args["max_new_tokens"] = 500

In [ ]:
output = pipe(prompt, **generation_args)
print(output[0]['generated_text'])

### 2.2 Agents Conversationnels (ChatBot)

In [110]:
messages = [
    {"role": "user", "content": "J'aimerais manger un fromage de Savoie, que me conseille tu ?"},
]

In [ ]:
output = pipe(messages, **generation_args)
print(output[0]['generated_text'])

In [112]:
messages.append({"role": "assistant", "content": output[0]['generated_text']})

In [113]:
messages.append({"role": "user", "content": "Je cherche un fromage plus insolite"})

In [ ]:
messages

In [ ]:
output = pipe(messages, **generation_args)
print(output[0]['generated_text'])

In [ ]:
messages = [
    {"role": "user", "content": "L'utilisateur est un amateur de fromage de longue date. Il attends de toi de découvrir des fromages peu connu et insolites. Acceuille le avec un message en français."}
]

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "do_sample": False,
    # pour utiliser le paramètre "temperature", passer le paramètre "do_sample" à True
    #"temperature": float(SEED)
}

usr_input = ""
while usr_input != "fin":
  output = pipe(messages, **generation_args)
  print("\nChatBot:", output[0]['generated_text'], "\n")
  usr_input = input("Utilisateur:")

  messages.append({"role": "assistant", "content": output[0]['generated_text']})
  messages.append({"role": "user", "content": usr_input})



### 2.3 Techniques de génération de textes (Prompt Engineering)

CoT: https://www.promptingguide.ai/techniques/cot

In [131]:
messages = [
    {
        "role": "user",
        "content":
        """
          Détaille ta réponse étape par étape.
          Je cherche un fromage de Savoie, que je conseille-tu?
        """
     }
]

In [ ]:
output = pipe(messages, **generation_args)
print(output[0]['generated_text'])

ToT: https://www.promptingguide.ai/techniques/tot

In [135]:
generation_args = {
    "max_new_tokens": 5000,
    "return_full_text": False,
    "do_sample": False,
    # pour utiliser le paramètre "temperature", passer le paramètre "do_sample" à True
    #"temperature": float(SEED)
}

messages = [
    {
        "role": "user",
        "content":
        """
          Imagine que trois experts différents débattent pour répondre à cette question.
          Chaque expert écrit une étape de sa réflexion et la partage avec le groupe.
          Puis tous les experts continue ainsi de suite, étape par étape.
          Si un expert réalise qu'il s'est trompé à un moment de sa réflexion, il sort de la discussion.
          La question est: "Je cherche un fromage de Savoie, que je conseille-tu?"
        """
     }
]

In [ ]:
output = pipe(messages, **generation_args)
print(output[0]['generated_text'])

GKP: https://www.promptingguide.ai/techniques/knowledge

In [137]:
messages = [
    {
        "role": "user",
        "content":
        """
          Avant de répondre à la question, écrit les différentes connaissances que tu as sur le sujet.
          La question est: "Je cherche un fromage de Savoie, que je conseille-tu?"
        """
     }
]

In [ ]:
output = pipe(messages, **generation_args)
print(output[0]['generated_text'])

In [139]:
messages = [
    {
        "role": "user",
        "content":
        """
          Avant de répondre à la question, commence par écrire les différentes connaissances que tu as sur le sujet.
          Ensuite, imagine que trois experts différents débattent pour répondre à cette question.
          Chaque expert écrit une étape de sa réflexion et la partage avec le groupe.
          Puis tous les experts continue ainsi de suite, étape par étape.
          Si un expert réalise qu'il s'est trompé à un moment de sa réflexion, il sort de la discussion.
          La question est: "Je cherche un fromage de Savoie, que je conseille-tu?"
        """
     }
]

In [ ]:
output = pipe(messages, **generation_args)
print(output[0]['generated_text'])

## Partie 3 - Combiner encodage et décodage avec les LLM Agents

TODO RAG